In [ ]:
import json, numpy as np, open3d as o3d

# ---------- 1. JSON 파라미터 읽기 ----------
with open("cylinder_fitting_result.json") as f:
    info = json.load(f)

assert info["type"] == "cylinder"
axis_dir = np.asarray(info["params"][0])
axis_pt  = np.asarray(info["params"][1])
radius   = float(info["params"][2])
axis_dir /= np.linalg.norm(axis_dir)

# ---------- 2. 실린더 높이 ----------
scan_xyz = np.loadtxt("cylinder.xyz")[:, :3]
proj = (scan_xyz - axis_pt) @ axis_dir
h_min, h_max = proj.min(), proj.max()
half_h = (h_max - h_min) / 2

# ---------- 3. 표면 점 샘플링 ----------
def sample_cylinder(center, direction, r, half_h,
                    max_points=15_000, base_theta=180):
    """
    실린더 표면을 최대 max_points 개 이하로 균일 샘플링.
    base_theta : 한 바퀴 각도 분할 기본값(>0)
    """
    # (1) 분할 수 계산
    n_theta = base_theta
    n_h     = int(max_points / n_theta)
    if n_h < 1:                       # 만약 너무 적으면
        n_h = 1
        n_theta = max_points

    # (2) 직교 벡터 u, v
    tmp = np.array([1,0,0]) if abs(direction[0]) < 0.9 else np.array([0,1,0])
    u = np.cross(direction, tmp);  u /= np.linalg.norm(u)
    v = np.cross(direction, u)

    zs      = np.linspace(-half_h, half_h, n_h, endpoint=True)
    thetas  = np.linspace(0, 2*np.pi, n_theta, endpoint=False)

    pts = [center + direction*z + r*np.cos(th)*u + r*np.sin(th)*v
           for z in zs for th in thetas]
    return np.vstack(pts)

# ---- 샘플링 실행 (최대 15 000 점) ----
cyl_xyz = sample_cylinder(axis_pt, axis_dir, radius, half_h,
                          max_points=15_000, base_theta=180)

print("샘플 점:", cyl_xyz.shape)      # (≤ 15000, 3)


샘플 점: (36000, 3)


In [8]:
# ---------- 4. z‑좌표(축 방향 거리) 계산 ----------
proj_cyl = (cyl_xyz - axis_pt) @ axis_dir          # (N,)
z_min, z_max = proj_cyl.min(), proj_cyl.max()
bins = np.linspace(z_min, z_max, 4)                # 3 구간 → 4 경계

labels = np.digitize(proj_cyl, bins) - 1           # 0,1,2

# ---------- 5. .xyzc 저장 ----------
xyzc = np.hstack([cyl_xyz, labels[:,None]])
np.savetxt("cylinder_synthetic_3cls.xyzc", xyzc,
           fmt="%.6f %.6f %.6f %d")
print("저장 완료 → cylinder_synthetic_3cls.xyzc")


저장 완료 → cylinder_synthetic_3cls.xyzc


In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class XYzcDataset(Dataset):
    """
    .xyzc 파일 : 4열 (x y z label) ASCII
    """
    def __init__(self, file_list, n_points=4096):
        self.files = file_list
        self.n_points = n_points

    def __len__(self):
        return len(self.files)

    def _load_xyzc(self, path):
        arr = np.loadtxt(path)               # (N,4)
        xyz  = arr[:, :3]
        label = arr[:, 3].astype(np.int64)
        return xyz, label

    def __getitem__(self, idx):
        xyz, label = self._load_xyzc(self.files[idx])

        # 랜덤 샘플링 / 패딩
        if xyz.shape[0] >= self.n_points:
            sel = np.random.choice(xyz.shape[0], self.n_points, replace=False)
        else:
            sel = np.random.choice(xyz.shape[0], self.n_points, replace=True)
        xyz  = xyz[sel]
        label = label[sel]

        # 중심화 & 정규화(단순 bounding box 스케일)
        xyz = xyz - xyz.mean(0, keepdims=True)
        scale = np.max(np.linalg.norm(xyz, axis=1))
        xyz = xyz / scale

        return torch.from_numpy(xyz).float(), torch.from_numpy(label)


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class PointNetSeg(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

        self.conv4 = nn.Conv1d(1088, 512, 1)
        self.conv5 = nn.Conv1d(512, 256, 1)
        self.conv6 = nn.Conv1d(256, 128, 1)
        self.conv7 = nn.Conv1d(128, num_classes, 1)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
        self.bn6 = nn.BatchNorm1d(128)

    def forward(self, x):           # x: (B,3,N)
        B, _, N = x.size()
        x1 = F.relu(self.bn1(self.conv1(x)))
        x2 = F.relu(self.bn2(self.conv2(x1)))
        x3 = F.relu(self.bn3(self.conv3(x2)))      # (B,1024,N)
        x_max = torch.max(x3, 2, keepdim=True)[0]  # (B,1024,1)
        x_max_repeat = x_max.repeat(1, 1, N)       # (B,1024,N)
        x_cat = torch.cat([x1, x_max_repeat], 1)   # (B,64+1024=1088,N)

        x = F.relu(self.bn4(self.conv4(x_cat)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.conv7(x)                          # (B,num_classes,N)
        return x


In [5]:
from tqdm import tqdm

# 파일 리스트 예시 (학습·검증)
train_files = ["data/cylinder_train1.xyzc", "data/cylinder_train2.xyzc"]
val_files   = ["data/cylinder_val.xyzc"]

train_ds = XYzcDataset(train_files, n_points=4096)
val_ds   = XYzcDataset(val_files,   n_points=4096)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = PointNetSeg(num_classes=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion  = nn.CrossEntropyLoss()

for epoch in range(50):
    model.train()
    running_loss = 0.0
    for xyz, label in tqdm(train_loader, desc=f"Epoch {epoch}"):
        xyz, label = xyz.to(device), label.to(device)     # xyz:(B,N,3)
        xyz = xyz.permute(0,2,1)                          # (B,3,N)
        optimizer.zero_grad()
        pred = model(xyz)                                 # (B,3,N)
        loss = criterion(pred, label)
        loss.backward(); optimizer.step()
        running_loss += loss.item()
    print(f"[Epoch {epoch}] train loss: {running_loss/len(train_loader):.4f}")

    # --- 간단 검증 ---
    model.eval(); correct, total = 0, 0
    with torch.no_grad():
        for xyz, label in val_loader:
            xyz, label = xyz.to(device), label.to(device)
            xyz = xyz.permute(0,2,1)
            pred = model(xyz).argmax(1)                   # (B,N)
            correct += (pred == label).sum().item()
            total   += label.numel()
    print(f"  val acc: {correct/total:.4f}")

# 가중치 저장
torch.save(model.state_dict(), "seg_cylinder_3cls.pth")


Epoch 0: 0it [00:00, ?it/s]


ZeroDivisionError: float division by zero

In [ ]:
import open3d as o3d

def infer_xyzc(model, xyzc_path, out_xyzc_path):
    xyzc = np.loadtxt(xyzc_path)
    xyz  = xyzc[:, :3]
    xyz_norm = xyz - xyz.mean(0, keepdims=True)
    scale = np.max(np.linalg.norm(xyz_norm, axis=1))
    xyz_norm /= scale

    with torch.no_grad():
        pts = torch.from_numpy(xyz_norm).float().unsqueeze(0).to(device)  # (1,N,3)
        pts = pts.permute(0,2,1)
        logits = model(pts).cpu().squeeze(0).transpose(0,1)               # (N,3)
        pred = logits.argmax(1).numpy()

    xyzc_out = np.hstack([xyz, pred[:,None]])
    np.savetxt(out_xyzc_path, xyzc_out, fmt="%.6f %.6f %.6f %d")
    print("saved:", out_xyzc_path)

    # 시각화용 PLY
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    color_map = np.array([[1,0,0],[0,1,0],[0,0,1]])
    pcd.colors = o3d.utility.Vector3dVector(color_map[pred])
    o3d.io.write_point_cloud(out_xyzc_path.replace(".xyzc",".ply"), pcd)

# 추론 실행
model.load_state_dict(torch.load("seg_cylinder_3cls.pth", map_location=device))
model.eval()
infer_xyzc(model, "data/cylinder_test.xyzc", "cylinder_pred.xyzc")
